### Advent of Code 

In [1]:
# Includes (Some includes inspired by Peter Norvig solution (https://github.com/norvig/pytudes))

import os
import urllib.request


def Inputstr(day, year=2017): 
    "The contents of this day's input file as a str."
    return Input(day, year).read().rstrip('\n')

def Input(day, year=2017):
    "Open the day's input file"
    directory = 'data/advent{}/day{}/'.format(year, day)
    filename = directory + 'input{}.txt'.format(day)
    return open(filename)

def mapt(fn, *arg):
    return tuple(map(fn, *arg))

def Atom(input):
    "Parse a str token into a number, or leave it as a str."
    try:
        return int(input)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return token
    
def Vector(lines):
    return mapt(Atom, lines.replace(',', ' ').split())

def Array(input):
    lines = input.splitlines()
    return mapt(Vector, lines)

def first(iterable, default = None):
     return next(iter(iterable), default)


In [2]:
assert mapt(int, '1234') == (1, 2, 3, 4)

assert Array('''5 1 9 5
7 5 3
2 4 6 8''') == (
                (5, 1, 9, 5), 
                (7, 5, 3),
                (2, 4, 6, 8)
               )

assert first('abc') == first(['a', 'b', 'c']) == 'a'


### --- Day 1: Inverse Captcha ---

The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

For example:

1122 produces a sum of 3 (1 + 2) because the first digit (1) matches the second digit and the third digit (2) matches the fourth digit.
1111 produces 4 because each digit (all 1) matches the next.
1234 produces 0 because no digit matches the next.
91212129 produces 9 because the only digit that matches the next one is the last digit, 9.



In [3]:
def inverse_captcha(input):
    d = input[0]
    n = len(input)
    s = 0
    for ad in input[1:]:
        if ad == d:
            s += (ord(ad) - 48)
        d = ad
        
    if input[n-1] == input[0]:
        s += (ord(input[n-1]) - 48)
    
    return s

input = Inputstr(1)
N = len(input)

print (inverse_captcha(input))

digits = mapt(int, input)
sum(digits[i] for i in range(N) if digits[i] == digits[i - 1])


1097


1097

#### Part 2

Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. That is, if your list contains 10 items, only include a digit in your sum if the digit 10/2 = 5 steps forward matches it. Fortunately, your list has an even number of elements.


In [4]:
def inverse_captcha_halfway(input):
    n = len(input)
    h = int (n / 2)
    the_sum = 0
    for i in range(n):
        if input[i] == input[ (i+h) % n]:
            the_sum += (ord(input[i]) - 48)
    
    return the_sum


input = Inputstr(1)
N = len(input)

print (inverse_captcha_halfway(input))

# Norvig version
digits = mapt(int, input)

sum(digits[i] for i in range(N) if digits[i] == digits[ i - N // 2])

1188


1188

### Day 2: Checksum

In [5]:
d = Array(Inputstr(2))

sum(max(t) - min(t) for t in d)

51139

### Part 2

It sounds like the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.



In [6]:
3 % 6

3

In [16]:
data = Array('''5 9 2 8
9 4 7 3
3 12 6 4
''')

# data = Array(Inputstr(2))

tsum = 0
for row in data:
    for i in range(len(row)):
        for j in range(i+1, len(row)):
            a, b = row[i], row[j]
            if a >= b and a % b == 0:
                tsum += (a // b)
            elif b % a == 0:
                tsum += (b // a)

print (tsum)    

# Norvig solution, however it only considers first pair that are divisible as whole. 
# Note: Just replacing first with sum in evendiv funciton fix this. 

def evendiv(row):
#     print(list(a // b for a in row for b in row if a > b and a // b == a / b))
    return sum(a // b for a in row for b in row if a > b and a // b == a / b)
    
sum(map(evendiv, data))

18
[4]
[3]
[4, 2, 3, 2]


18

### Day 3 Spiral Memory

You come across an experimental new kind of memory stored on an infinite two-dimensional grid.

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

<pre>
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
</pre>

While this is very space-efficient (no squares are skipped), requested data must be carried back to square 1 (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the Manhattan Distance between the location of the data and square 1.

For example:

* Data from square **1** is carried **0 steps**, since it's at the access port.
* Data from square **12** is carried **3 steps**, such as: down, left, left.
* Data from square **23** is carried only **2 steps**: up twice.
* Data from square **1024** must be carried **31 steps**.

How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port?

In [ ]:
goal = 23

navs = ((1, 0), (0, 1), (-1, 0), (0, -1))
limit = 1
cnt = 0

start = (0, 0)
node = 1

while node != goal:
    for nav in navs:
        x, y = nav
        cx, cy = start
        i = 0
        while i < limit:
            cx, cy = cx + x, cy + y
            i += 1
            node += 1
            
            